Tree Augmented Bayesian Network Classifier

In [1]:
from collections import deque
from math import log
import numpy as np
import sys
from sklearn.model_selection import train_test_split

The function calculates the Conditional Mutual Information between attributes given the class variable

In [2]:
def get_CMI(f1, f2, values1, values2, y_values, X, Y):
    
    Ip = 0
    samples = len(X)
    
    for y in y_values:
        for v1 in values1:
            for v2 in values2:
                Pxyz = 0
                Pxy_z = 0
                Px_z = 0
                Py_z = 0
                totz = 0
                
                for i in range(samples):
                    if X[i][f1] == v1 and X[i][f2] == v2 and Y[i] == y:
                        Pxyz += 1
                    
                    if Y[i] == y:
                        totz+=1
                        if X[i][f1] == v1:
                            Px_z += 1
                        if X[i][f2] == v2:
                            Py_z += 1
                
                if totz > 0:         
                    Pxy_z = Pxyz/totz
                    Px_z /=totz
                    Py_z /=totz
                Pxyz /= samples
                if Px_z and Py_z and Pxy_z:
                    Ip += Pxyz*log(Pxy_z/(Px_z*Py_z))
                
    return Ip

    
    

Kruskals Algorithm to find the Maximum Spanning Tree

In [3]:
def kruskals(edges):
    
    visited = set()
    adjacency = []
    
    for edge in edges:
        if edge[1] not in visited or edge[2] not in visited:
            visited.add(edge[1])
            visited.add(edge[2])
            adjacency.append([edge[1], edge[2]])
            
    return adjacency
    


Training the Bayesian Network Classifier

In [4]:
def train(X,Y):
    features = len(X[0])
    samples = len(X)
    
    attributes = [set() for _ in range(features)]
    y_values = set(Y)
    for feature in range(features):
        for instance in range(samples):
            # print(instance, feature)
            
            attributes[feature] = X[instance][feature]
    
    CMIs = []
    
    # Step 1 and 2: Computing Ip and making the graph (CMI) 
    for feature1 in range(features):
        for feature2 in range(feature1+1, features):
            CMIs.append((get_CMI(feature1, feature2, attributes[feature1], attributes[feature2], y_values, X, Y), feature1, feature2))
        
    CMIs.sort(reverse=True)
    
    # Step 3: Building the maximum spanning tree
    max_sp_tree = kruskals(CMIs)
    
    # Step 4: Selecting first feature (feature = 0) as the root node
    Bayesian_Network = [[] for feature in range(features)]
    
    adj = [[] for feature in range(features)]
    for edge in max_sp_tree:
        adj[edge[0]].append(edge[1])
        adj[edge[1]].append(edge[0])
        
    # Performing BFS to build the Bayesian Network
    q = deque()
    visited = set()
    q.append(0) # Pushing first feature
    visited.add(0)
    
    while len(q):
        node = q.popleft()
        
        for neighbour in adj[node]:
            
            if neighbour not in visited:
                Bayesian_Network[node].append(neighbour)
                q.append(neighbour)
                visited.add(neighbour)
                
    # Step 5: Constructing TAN model
    parents = [[] for feature in range(features)]
    for feature in range(features):
        for child in Bayesian_Network[feature]:
            parents[child].append(feature)
            
    return parents
    

Prediction on a test point using the built Bayesian Network

In [5]:
def predict(xtest, parents, X, Y):
    y_values = set(Y)
    probs = {}
    features = len(X[0])
    samples = len(X)
    
    for y in y_values:
        prob = 1
        for feature in range(features):
            
            count = 0
            tot = 0
            for i in range(samples):
                
                if y == Y[i]:
                    consider = True
                    for p in parents[feature]:
                        if X[i][p] != xtest[p]:
                            consider = False
                            break
                    if consider:
                        tot += 1
                        if X[i][feature] == xtest[feature]:
                            count += 1
            prob *= (count/tot)
        
        count = 0
        for i in range(samples):
            if y == Y[i]:
                count += 1
                
        prob*=(count/samples)
        probs[y] = prob
    
    # Using probability of each label to predict the actual label
    label = Y[0]
    max_prob = probs[Y[0]]
    for key in probs:
        if probs[key] > max_prob:
            max_prob = probs[key]
            label = key
            
    return label

Note: We can use the same predict function for Naive Bayes. The only difference is that the parents list will be an empty list of size 'number of features' because naive bayes assumes conditional independence over C, therefore, the tree will have C as the root node and all the attributes (Ai) as the children of C.

In [6]:

def generate_dataset(path):
    fp = open(path, 'r')
    lines = fp.readlines()
    data = []
    print("Total number of samples:", len(lines))
    for line in lines:
        data.append(line[:-1].split(','))
        
    fp.close()
    X, Y = [], []
    for i in data:
        X.append(i[:-1])
        Y.append(i[-1])
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=2)
    return X_train, X_test, Y_train, Y_test
       

In [8]:
        
#path = 'chess dataset/kr-vs-kp.txt'
# path = 'heart dataset/data.txt'
path = 'kr-vs-kp.txt'
X_train, X_test, Y_train, Y_test = generate_dataset(path)
print('Training Dataset:',len(X_train), '|', 'Testing Dataset:',len(X_test))
TAN_Parents = train(X_train, Y_train)
# print(BN_Parents)
NB_Parents = [[] for _ in range(len(X_train[0]))] # Parents list for Naive Bayes is empty because we assume conditional independence
accuracyBN = 0
accuracyNB = 0
for t in range(len(X_test)):
    accuracyBN += predict(X_test[t], TAN_Parents, X_train, Y_train) == Y_test[t]
    accuracyNB += predict(X_test[t], NB_Parents, X_train, Y_train) == Y_test[t]
print('TAN Accuracy:', accuracyBN/len(X_test)*100, '|','Naive Bayes Accuracy:',accuracyNB/len(X_test)*100)
    

Total number of samples: 3196
Training Dataset: 2556 | Testing Dataset: 640
TAN Accuracy: 91.71875 | Naive Bayes Accuracy: 90.0
